# SASPy Example
## 1. Import Modules

In [ ]:
# Basic modules
import pandas
import numpy
from getpass import getuser, getpass
import datetime as dt

# SASPy
import saspy

# Display Options
from IPython.display import HTML
pd.set_option('display.max_columns', None)

## 2. Setup SAS Connection

In [ ]:
username = getuser()
sas_session = saspy.SASsession(cfgfile=f"C:\\Users\\{username}\\.saspy\\sascfg_personal.py")

## 3. SAS Enterprise Guide Variables

In [ ]:
# Send to SAS
setup_query = """
* Create _CLIENTUSERID. macro var;
%let _CLIENTUSERID. = "&_METAUSER.";
"""

setup_results = sas_session.submit(setup_query)

# To view results
display(HTML(setup_results['LST']))

# To view log
print(setup_results['LOG'])
print(sas_session.lastlog())

## 4. `df2sd` Parameters
Create dictionaries to store setup parameters to upload dataframe to SAS

In [ ]:
dsopts = {
    'format': {
        'datadate': 'YYMMDDN8.'
        }
    }

_datetimes = {
    'datevar' : 'date'
    }

_outfmts = {
    'datevar' : 'DATE9.'
    }

## 5. SAS Magic Command

In [ ]:
%%SAS sas_session
proc sql outobs=5;
    select * from sashelp.cars;
quit;

## 6. `sasdata2dataframe` and `df2sd`
This method enables us to retrieve data from SAS server into a Python dataframe.

`sasdata2dataframe` downloads data locally.

`df2sd` uploads data to the SAS server.

In [ ]:
# Download data
df = sas_session.sasdata2dataframe(
    table='cars',
    libref='sashelp'
    )

In [ ]:
# Upload data to SAS
sas_session.df2sd(
    df = df,
    table = 'cars',
    libref = 'work',
    datetimes = _datetimes,
    outfmts = {
        **{
            # Add other formats here
        },
        **_outfmts
        }
    )